In [33]:
import pandas as pd
import geopandas as gpd
import os
import sys
import pylab
import re
import urllib.request as request
from datetime import datetime
from bs4 import BeautifulSoup

% pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


# Collect data January turnstile data (2017 & 2016)

In [19]:
MTA_TURNSTILE_URL = "http://web.mta.info/developers/turnstile.html"
MTA_FILE_ROOT_URL = "http://web.mta.info/developers/"

In [35]:
'''
slightly modified from https://github.com/piratefsh/mta-turnstile-scraper/blob/master/mta_scraper.py'''

def get_site():
    f = request.urlopen(MTA_TURNSTILE_URL)
    content = f.read()
    return content

def get_turnstile_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a')
    turnstile_links = [(link.text, MTA_FILE_ROOT_URL + link['href']) for link in links if re.match('.*day.*20..', link.text)]
    return turnstile_links 

def get_links_by_date(start, end):
    links = get_links()
    in_range = []
    for text,link in links:
        date = datetime.datetime.strptime(text , "%A, %B %d, %Y") 
        if date >= start and date <= end:
            in_range.append((text, link))
    
    return in_range

def get_links():
    return get_turnstile_links(get_site())

In [39]:
a = get_site()
b = get_turnstile_links(a)
start = datetime.datetime(2016, 1, 1)
end = datetime.datetime(2016, 2, 1)
desired = get_links_by_date(start, end)

In [120]:
metadata = 'http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description_pre-10-18-2014.txt'
f = request.urlopen(metadata)
content = f.readlines()
col_names_14 = str(content[2]).split(',')

def turnstile_df(links, post2014 = True): 
    '''takes links and returns dataframe with all data in one
    pre 2014 was captured differently and thus requires a second argument for correct columns'''
    df = pd.DataFrame()
    if post2014 == True: 
        for text, link in links: 
            temp_df = pd.read_csv(link, parse_dates = True, infer_datetime_format = True)
            df = df.append(temp_df)
    elif post2014 == False: 
        for text, link in links: 
            temp_df = pd.read_csv(link, header = None, names = col_names_14, parse_dates = True,
                                 infer_datetime_format = True)
            df = df.append(temp_df)
    df.reset_index(inplace = True)
    return df

In [52]:
jan2016 = turnstile_df(desired)

In [53]:
a = get_site()
b = get_turnstile_links(a)
start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2017, 2, 1)
desired = get_links_by_date(start, end)
jan2017 = turnstile_df(desired)

In [54]:
a = get_site()
b = get_turnstile_links(a)
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2015, 2, 1)
desired = get_links_by_date(start, end)
jan2015 = turnstile_df(desired)

In [126]:
a = get_site()
b = get_turnstile_links(a)
start = datetime.datetime(2014, 1, 1)
end = datetime.datetime(2014, 2, 1)
desired = get_links_by_date(start, end)
jan2014 = turnstile_df(desired, False)

In [127]:
a = get_site()
b = get_turnstile_links(a)
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2013, 2, 1)
desired = get_links_by_date(start, end)
jan2013 = turnstile_df(desired, False)

In [128]:
a = get_site()
b = get_turnstile_links(a)
start = datetime.datetime(2012, 1, 1)
end = datetime.datetime(2012, 2, 1)
desired = get_links_by_date(start, end)
jan2012 = turnstile_df(desired, False)

# Data Cleaning: Remove all data outside the range of January
# Desired Stations: Q 72, 86, 96
# Desired Stations: 456 - 68, 77, 86, 96

In [202]:
def mta_cleaning(df):
    # remove out of bounds dates and stations
    irt_stations = ['68ST-HUNTER CO', '77 ST', '86 ST', '96 ST']
    irt_line = '6'
    bmt_stations = ['72 ST-2 AVE', '86 ST-2 AVE', '96 ST-2 AVE']
    bmt_line = 'Q'
    
    irt_df = df[df.LINENAME.str.contains(irt_line)][df.STATION.isin(irt_stations)]
    bmt_df = df[df.LINENAME.str.contains(bmt_line)][df.STATION.isin(bmt_stations)]
    
    UES_df = irt_df.append(bmt_df)
    
    UES_df.DATE = UES_df.DATE.apply(pd.to_datetime)
    
    jan_mask = UES_df.DATE.map(lambda x: x.month) == 1
    
    return UES_df[jan_mask]

In [203]:
UES_jan2017 = mta_cleaning(jan2017)
UES_jan2016 = mta_cleaning(jan2016)
UES_jan2015 = mta_cleaning(jan2015)

C:\Users\shibbie\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\shibbie\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [205]:
UES_jan2017.to_csv('UES_jan2017')
UES_jan2016.to_csv('UES_jan2016')
UES_jan2015.to_csv('UES_jan2015')